In [ ]:
import torch
import torch.autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import numpy as np

class MLP(nn.Module):
    def __init__(self, size_list, dropout = False, dropoutProb = 0.1, batchNorm = False):
        super(MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.ReLU())
            
            if batchNorm:
                layers.append(nn.BatchNorm1d(size_list[i+1]))
                
            if dropout:
                layers.append(nn.Dropout(p = dropoutProb))
            
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        
        # Unpack the list
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

class RNN(nn.Module):
    def __init__(self, size_list):
        super(RNN, self).__init__()
        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=size_list[0],
            hidden_size=size_list[1],         # rnn hidden unit
            num_layers=len(size_list)-2,           # number of rnn layer
            )

        self.out = nn.Linear(size_list[-2], size_list[-1])

        

    def forward(self, x):
        
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return out
    

def train_epoch(model, optimizer, X_train, y_train, criterion):
    model.train()
    
    start_time = time.time()

    optimizer.zero_grad()  

    outputs = model(X_train)
    
    loss = criterion(outputs, y_train)
    
    running_loss = loss.item()

    loss.backward()
    
    optimizer.step()
    
    end_time = time.time()
    
    #print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss




# Optimizer can be 'SGD', 'RMSprop', 'ADAM', 
def nnTrain(X_train, y_train, size_list, dropout = False, dropoutProb = 0.1, batchNorm = False, 
                optimizer = 'SGD', lr = 0.01, n_epochs = 100, LSTM = False):   

    X_train = torch.autograd.Variable(torch.Tensor(X_train.values.astype(float)))
    y_train = torch.autograd.Variable(torch.Tensor(y_train.values.astype(float)))
    
    if LSTM == True:
        X_train = X_train.view(-1, X_train.shape[0], X_train.shape[1])
        model = RNN(size_list)
    else:
        model = MLP(size_list, dropout, dropoutProb, batchNorm)
    
    
    criterion = nn.MSELoss()
    
    if optimizer == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr = lr)
    if optimizer == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr = lr)
    if optimizer == 'ADAM':
        optimizer = optim.ADAM(model.parameters(), lr = lr)
    
    Train_loss = []
    
    for i in range(n_epochs):
        train_loss = train_epoch(model, optimizer, X_train, y_train, criterion)
        Train_loss.append(train_loss)
    
    return model, Train_loss


def nnTest(model, X_test, y_test):
    X_test = torch.autograd.Variable(torch.Tensor(X_test.values.astype(float)))
    y_test = torch.autograd.Variable(torch.Tensor(y_test.values.astype(float)))
    criterion = nn.MSELoss()
    
    if hasattr(model, 'rnn'):
        X_test = X_test.view(-1, X_test.shape[0], X_test.shape[1])
    
    with torch.no_grad():
        model.eval()        

        outputs = model(X_test)

        loss = criterion(outputs, y_test).detach()
        running_loss = loss.item()
        
        return running_loss
    
def nnPredict(model, X_test):
    X_test = torch.autograd.Variable(torch.Tensor(X_test.values.astype(float)))
    
    if hasattr(model, 'rnn'):
        X_test = X_test.view(-1, X_test.shape[0], X_test.shape[1])
    
    with torch.no_grad():
        model.eval()        
        outputs = model(X_test)
    return np.array(outputs).flatten()

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import pandas_datareader as web
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import neural_network as npy
import os 
import sys


def get_random_tickers(n, ticklist):
    if n >len(ticklist):
        raise Exception('n is bigger than ticklist')
    return np.random.choice(ticklist, size=n, replace=False)
RSVE = []


# Our initial ticklist
ticklist = ['ADS.DE', 'ALV.DE', 'BAS.DE', 'BEI.DE', 'BMW.DE', 'CON.DE', 'DAI.DE', 'DBK.DE', 'DTE.DE', 'EOAN.DE', 'FME.DE',
 'FRE.DE', 'HEI.DE', 'HEN3.DE', 'LHA.DE', 'LIN.DE', 'MRK.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'TKA.DE', 'VOW3.DE']

# The 7 tickers chosen randomly from this ticklist

ticklist = get_random_tickers(10, ticklist)
#ticklist = ['LIN.DE', 'HEN3.DE', 'DAI.DE', 'FME.DE', 'MUV2.DE', 'SIE.DE', 'DBK.DE']
ticklist = ['CON.DE', 'RWE.DE', 'DTE.DE', 'BEI.DE', 'HEI.DE', 'LIN.DE','FRE.DE', 'ADS.DE','FME.DE','MRK.DE']

tickdict = dict(zip(ticklist, range(1,len(ticklist)+1)))

## Paramteres
d0 = '2001-01-01' # begining of the waiting period
d1 = '2004-01-01' # end of the CV period - beg
d2 = '2006-01-01' # begining of the test period
d3 = '2008-01-01' # end of the test period

dcv1 = '2004-01-01'
dcv2 = '2005-01-01'


### Parameters
norm=True
cv_nlayers=True
cv_nneurones = False
lbd = 0.2 # history weight metric
alpha = 0.09 # risk management metric
cst = 10 #weight metric
delta = 0.91



nneurones = 10
nlayers = 4
drp = False
drpProb = 0.1
batch = False 
opt = 'SGD'
learning_rate = 0.01
n_epochs = 400



# Generate the X matrix and Y matrix and make them have only trading days

dtes = pd.read_csv('trading_days.csv', index_col=0)
tempdt = dtes.copy()
tempdt.set_index('Buy', drop=True, inplace=True)
tempdt.index = pd.to_datetime(tempdt.index)

Y = []
X = pd.DataFrame(columns=['Date', 'EMA10', 'EMA16', 'EMA22', 'SMA10', 'SMA16', 'SMA22','ValueAtRisk', 
                       'Bollu20', 'Bollu26', 'Bollu32', 'Bolld20', 'Bolld26', 'Bolld32',
                       'Mom12', 'Mom18', 'Mom24', 'ACC12', 'ACC18', 'ACC24', 'ROC10', 'ROC16',
                       'ROC22', 'MACD1812', 'MACD2412','MACD3012', 'MACDS18129', 'MACDS24129', 'MACDS30129', 
                       'RSI8', 'RSI14', 'RSI20', 'OBV', 'CHV1010', 'CHV1016', 'CHV1022',
                       'FastK12', 'FastD12', 'FastK18', 'SlowK12', 'FastD18', 'SlowD12',
                       'FastK24', 'SlowK18', 'FastD24', 'SlowD18', 'SlowK24', 'SlowD24',
                       'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 
                       'Ticker','Month', 'DAX', 'ADL', 'Type1', 'Type2', 'Type3', 'Y'])

dax = web.get_data_yahoo('^GDAXI', start=d0, end=d3)
prices = pd.DataFrame(columns=['Ticker', 'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Date'])


for tick in ticklist:
    if norm:
        temp = pd.read_csv('tickDataNorm/'+ str(delta).replace('.', '') +'/' + tick.replace('.', '') +'.csv', index_col=0)
    else:
        temp = pd.read_csv('tickData/'+ str(delta).replace('.', '') +'/' + tick.replace('.', '') +'.csv', index_col=0)
    prices = pd.concat([prices, temp.loc[:, ['Ticker', 'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Date']]],axis=0, ignore_index=True, sort=False,copy=True)
    # Select dates around events
    temp.set_index('Date', inplace=True, drop=True)
    temp.index = pd.to_datetime(temp.index)
    B = temp.loc[pd.to_datetime(dtes.Buy), 'Close']
    S = temp.loc[pd.to_datetime(dtes.Sell), 'Close']
    temp = temp.loc[tempdt.index, :]
    temp['Y'] = 100*(S.values-B.values)/B.values
    
    mask = np.logical_not(np.isnan(temp['Y'].values))
    temp = temp.loc[mask, :]
    if norm:
        temp.loc[:,'High'] = temp.loc[:,'Norm_High']
        temp.loc[:,'Low'] = temp.loc[:,'Norm_Low']
        temp.loc[:,'Open'] = temp.loc[:,'Norm_Open']
        temp.loc[:,'Close'] = temp.loc[:,'Norm_Close']
        temp.loc[:,'AdjClose'] = temp.loc[:,'Norm_AdjClose']
    #temp = temp.loc[pd.to_datetime(dtes.Buy), :]
    temp['Month'] = temp.index.month
    temp['Date'] = temp.index
    temp['DAX'] = dax.loc[temp.index, 'Adj Close']
    temp.loc[:,'Type'] = tempdt.loc[mask, 'Type'].values
    temp = temp.loc[:,['Date', 'EMA10', 'EMA16', 'EMA22', 'SMA10', 'SMA16', 'SMA22','ValueAtRisk', 
                       'Bollu20', 'Bollu26', 'Bollu32', 'Bolld20', 'Bolld26', 'Bolld32',
                       'Mom12', 'Mom18', 'Mom24', 'ACC12', 'ACC18', 'ACC24', 'ROC10', 'ROC16',
                       'ROC22', 'MACD1812', 'MACD2412','MACD3012', 'MACDS18129', 'MACDS24129', 'MACDS30129', 
                       'RSI8', 'RSI14', 'RSI20', 'OBV', 'CHV1010', 'CHV1016', 'CHV1022',
                       'FastK12', 'FastD12', 'FastK18', 'SlowK12', 'FastD18', 'SlowD12',
                       'FastK24', 'SlowK18', 'FastD24', 'SlowD18', 'SlowK24', 'SlowD24',
                       'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Ticker',
                       'Month', 'DAX', 'ADL','Type', 'Y']]
    

    temp['Type1'] = (temp.loc[:,'Type'].values == 1)*1
    temp['Type2'] = (temp.loc[:,'Type'].values == 2)*1
    temp['Type3'] = (temp.loc[:,'Type'].values == 3)*1
    temp = temp.loc[:,['Date', 'EMA10', 'EMA16', 'EMA22', 'SMA10', 'SMA16', 'SMA22','ValueAtRisk', 
                       'Bollu20', 'Bollu26', 'Bollu32', 'Bolld20', 'Bolld26', 'Bolld32',
                       'Mom12', 'Mom18', 'Mom24', 'ACC12', 'ACC18', 'ACC24', 'ROC10', 'ROC16',
                       'ROC22', 'MACD1812', 'MACD2412','MACD3012', 'MACDS18129', 'MACDS24129', 'MACDS30129', 
                       'RSI8', 'RSI14', 'RSI20', 'OBV', 'CHV1010', 'CHV1016', 'CHV1022',
                       'FastK12', 'FastD12', 'FastK18', 'SlowK12', 'FastD18', 'SlowD12',
                       'FastK24', 'SlowK18', 'FastD24', 'SlowD18', 'SlowK24', 'SlowD24',
                       'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 
                       'Ticker','Month', 'DAX', 'ADL', 'Type1', 'Type2', 'Type3', 'Y']]
    print(temp.Ticker.unique())
    X = pd.concat([X, temp], axis=0, ignore_index=True, copy=True, sort=False)


In [ ]:
T = X.Ticker.unique()
tickdict = dict(zip(T, range(len(T))))
for tick in T:
    X.loc[X.Ticker==tick,'Ticker']= tickdict[tick]
    
X.sort_values(by=['Date', 'Ticker'], inplace=True)
X.set_index('Date', drop=True, inplace=True)
X = X.loc[((X.index>=pd.to_datetime(d0)) & (X.index<=pd.to_datetime(d3))), :]

## X is generated

## Normalizing features
C = ['EMA10', 'EMA16', 'EMA22', 'SMA10', 'SMA16', 'SMA22','ValueAtRisk', 
                       'Bollu20', 'Bollu26', 'Bollu32', 'Bolld20', 'Bolld26', 'Bolld32',
                       'Mom12', 'Mom18', 'Mom24', 'ACC12', 'ACC18', 'ACC24', 'ROC10', 'ROC16',
                       'ROC22', 'MACD1812', 'MACD2412','MACD3012', 'MACDS18129', 'MACDS24129', 'MACDS30129', 
                       'RSI8', 'RSI14', 'RSI20', 'OBV', 'CHV1010', 'CHV1016', 'CHV1022',
                       'FastK12', 'FastD12', 'FastK18', 'SlowK12', 'FastD18', 'SlowD12',
                       'FastK24', 'SlowK18', 'FastD24', 'SlowD18', 'SlowK24', 'SlowD24',
                       'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose', 'Ticker',
                       'Month', 'DAX', 'ADL','Type1', 'Type2', 'Type3']

Cnorm = ['EMA10', 'EMA16', 'EMA22', 'SMA10', 'SMA16', 'SMA22','ValueAtRisk', 
                       'Bollu20', 'Bollu26', 'Bollu32', 'Bolld20', 'Bolld26', 'Bolld32',
                       'Mom12', 'Mom18', 'Mom24', 'ACC12', 'ACC18', 'ACC24', 'ROC10', 'ROC16',
                       'ROC22', 'MACD1812', 'MACD2412','MACD3012', 'MACDS18129', 'MACDS24129', 'MACDS30129', 
                       'RSI8', 'RSI14', 'RSI20', 'OBV', 'CHV1010', 'CHV1016', 'CHV1022',
                       'FastK12', 'FastD12', 'FastK18', 'SlowK12', 'FastD18', 'SlowD12',
                       'FastK24', 'SlowK18', 'FastD24', 'SlowD18', 'SlowK24', 'SlowD24',
                       'High', 'Low', 'Open', 'Close', 'Volume', 'AdjClose',
                       'Month', 'DAX', 'ADL']


In [ ]:
# Remove outliers
q1, q2 = X['Y'].quantile(0.98), X['Y'].quantile(0.02)
X = X[(X['Y'] < q1) & (X['Y'] > q2)]

## Normalization
Xtrain1 = X.loc[((X.index>pd.to_datetime(d0)) & (X.index<=pd.to_datetime(d1))), C].copy()
Ytrain1 = X.loc[((X.index>pd.to_datetime(d0)) & (X.index<=pd.to_datetime(d1))), 'Y'].copy()

Xmean = np.mean(Xtrain1.loc[:,Cnorm])
Xstdev = np.std(Xtrain1.loc[:,Cnorm])



X_old = X.copy()
X.loc[:, Cnorm] = (X.loc[:, Cnorm]-Xmean)/(Xstdev)

Xcv1 = X.loc[((X.index>pd.to_datetime(d0)) & (X.index<=pd.to_datetime(d1))), C].copy()
Ycv1 = X.loc[((X.index>pd.to_datetime(d0)) & (X.index<=pd.to_datetime(d1))), 'Y'].copy()
Xtrain1 = X.loc[((X.index>pd.to_datetime(d1)) & (X.index<=pd.to_datetime(d2))), C].copy()
Ytrain1 = X.loc[((X.index>pd.to_datetime(d1)) & (X.index<=pd.to_datetime(d2))), 'Y'].copy()


# Vanilla Neural Network

In [ ]:
drp = False
drpProb = 0.1
batch = False
optimizer = 'SGD'
learning_rate = 0.02
n_epochs = 1000


layer_ls = list(range(1,8))
neuron_ls = list(range(1,16))


loss_array = np.zeros((len(layer_ls), len(neuron_ls)))
for nlayers in layer_ls:
    for nneurons in neuron_ls:
        ls = [nneurons]*nlayers
        ls = np.insert(ls, 0, Xcv1.shape[1]).tolist()
        ls.append(1)
        nnt, err = npy.nnTrain(Xtrain1, Ytrain1, ls, drp, drpProb, batch, opt, learning_rate, n_epochs) 
        loss_array[nlayers-1,nneurons-1]  = npy.nnTest(nnt, Xcv1, Ycv1)        
        plt.plot(err)        

plt.xlabel('Reps')
plt.ylabel('Losses')
plt.title('Computed train losses')
plt.show()

We see for each network, the loss converges before 1000 training epochs. 

In [ ]:
from matplotlib import cm

loss_df = pd.DataFrame(loss_array, index=layer_ls, columns=neuron_ls)
loss_df

from mpl_toolkits.mplot3d import Axes3D
neuron_array, layer_array = np.meshgrid(neuron_ls, layer_ls)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(layer_array, neuron_array, loss_array, cmap=cm.coolwarm)
ax.set_xlabel('layers')
ax.set_ylabel('neurons')
ax.set_zlabel('loss')
plt.show()

When increasing nlayers and nneurons do not improve the performance too much, we choose not to add too much complexity to the model. The selected number of layers is 4, and number of neurons per layer is 12.

# Improved Neural Network

We see the train errors converge on a level far lower the testing error. This is a sign of overfitting, so we try dropout. In addition, the optimization may fall into local minimum, so we try RMSprop and Adam. 

To control the covariate shift in the financial data, we also apply batch normalization

In [ ]:
param_dist = {'drp_ls': [True, False],
             'drpProb_ls': [0.3, 0.5 ,0.6, 0.7, 0.9],
             'batch_ls': [True, False],
             'optimizer_ls': ['SGD', 'RMSprop', 'ADAM'],
             'layer_ls': [2,4,6],
              'neuron_ls': [5,7,9,11]}



learning_rate = 0.02
n_epochs = 1000



result = {}
result2 = {}
for drp in param_dist['drp_ls']:
    for drpProb in param_dist['drpProb_ls']:
        for batch in param_dist['batch_ls']:
            for optimizer in param_dist['optimizer_ls']:
                for nlayer in param_dist['layer_ls']:
                    for nneurons in param_dist['neuron_ls']:
                        ls = [nneurons]*nlayers
                        ls = np.insert(ls, 0, Xcv1.shape[1]).tolist()
                        ls.append(1)
                        nnt, err = npy.nnTrain(Xtrain1, Ytrain1, ls, drp, drpProb, batch, opt, learning_rate, n_epochs) 
                        string = 'drp:' + str(drp) + ' dropProb:' + str(drpProb) + ' batch:' + str(batch)\
                        + ' optimizer:' + str(optimizer) + ' nlayer:' + str(nlayer) + ' nneurons:' + str(nneurons)
                        result[string]  = npy.nnTest(nnt, Xcv1, Ycv1) 
                        result2[string] = err
                        if sum(err>20) == 0:
                            plt.plot(err)
                        print(string, result[string])

plt.xlabel('Reps')
plt.ylabel('Losses')
plt.title('Computed train losses')
plt.show()

In [ ]:
param_dist = {'drp_ls': [True, False],
             'drpProb_ls': [0.3, 0.5 ,0.6, 0.7, 0.9],
             'batch_ls': [True, False],
             'optimizer_ls': ['SGD', 'RMSprop', 'ADAM'],
             'layer_ls': [2,4,6],
              'neuron_ls': [5,7,9,11]}



learning_rate = 0.02
n_epochs = 1000



result = {}
result2 = {}
for drp in param_dist['drp_ls']:
    for drpProb in param_dist['drpProb_ls']:
        for batch in param_dist['batch_ls']:
            for optimizer in param_dist['optimizer_ls']:
                for nlayer in param_dist['layer_ls']:
                    for nneurons in param_dist['neuron_ls']:
                        ls = [nneurons]*nlayers
                        ls = np.insert(ls, 0, Xcv1.shape[1]).tolist()
                        ls.append(1)
                        nnt, err = npy.nnTrain(Xtrain1, Ytrain1, ls, drp, drpProb, batch, opt, learning_rate, n_epochs) 
                        string = 'drp:' + str(drp) + ' dropProb:' + str(drpProb) + ' batch:' + str(batch)\
                        + ' optimizer:' + str(optimizer) + ' nlayer:' + str(nlayer) + ' nneurons:' + str(nneurons)
                        result[string]  = npy.nnTest(nnt, Xcv1, Ycv1) 
                        result2[string] = err
                        if sum(np.array(err)>20) == 0:
                            plt.plot(err)
                        print(string, result[string])

plt.xlabel('Reps')
plt.ylabel('Losses')
plt.title('Computed train losses')
plt.show()